In [2]:
import numpy as np
import pandas as pd
import cooler
from scipy.signal import find_peaks
from scipy.stats import norm
import xarray as xr
import glob

In [3]:
ct = group_name
#ct = 'Astro-NT_NN'

In [4]:
indir = '/data/female-amb/scDomain/'
res = 25000

In [5]:
chrom_size_path = '/ref/m3C/mm10.main.nochrM.nochrY.chrom.sizes'
chrom_sizes = cooler.read_chromsizes(chrom_size_path, all_names=True)

In [6]:
leg = [f"{ct}.{age}" for age in ['8wk','9mo','18mo']]
leg

['Astro-NT_NN.8wk', 'Astro-NT_NN.9mo', 'Astro-NT_NN.18mo']

In [7]:
from statsmodels.sandbox.stats.multicomp import multipletests as FDR
from scipy.stats import chi2_contingency

def diff_bound(bound_count_ct, cell_count_ct):
    tmp = cell_count_ct[:,None] - bound_count_ct
    stats = np.zeros(bound_count_ct.shape[1])
    pv = np.ones(bound_count_ct.shape[1])
    binfilter = np.logical_and(bound_count_ct.sum(axis=0)>0, tmp.sum(axis=0)>0)
    for i in range(bound_count_ct.shape[1]):
        if binfilter[i]:
            contig = [bound_count_ct[:,i], tmp[:,i]]
            stats[i], pv[i], _, _ = chi2_contingency(contig)
    fdr = FDR(pv, 0.01, 'fdr_bh')[1]
    return stats, pv

def shuffle_ct(i):
    global cell_count_ct, sc_border, leg
    np.random.seed(i)
    label = np.random.permutation(sc_border.obs[f'{ct_key}'])
    bound_count_ct = np.array([sc_border.X[label==xx].getnnz(axis=0) for xx in leg])
    bound_prob_ct = bound_count_ct / cell_count_ct[:,None]
    return diff_bound(bound_count_ct, cell_count_ct)[0]

def diff_bound_bulk(ins_count):
    stats = np.zeros(ins_count.shape[2])
    pv = np.ones(ins_count.shape[2])
    binfilter = (ins_count.min(axis=(0,1))>0)
    for i in range(ins_count.shape[2]):
        if binfilter[i]:
            stats[i], pv[i], _, _ = chi2_contingency(ins_count[:,:,i])
    fdr = FDR(pv, 0.01, 'fdr_bh')[1]
    return stats, pv

In [8]:
bound_count_ct = pd.read_hdf(f'{ct}_boundcount.hdf', key='data').loc[leg]
cell_count_ct = pd.read_csv(f'{ct}_cellcount.csv.gz', index_col=0, header=0)['count']
bound_prob_ct = (bound_count_ct / cell_count_ct.to_numpy()[:,None]).T

In [9]:
all_ncs  =glob.glob(f'{indir}/*.nc')

In [11]:
ins_count = xr.open_dataset(f'/home/qzeng_salk_edu/project/240205-domain/hicluster_bulk_domain/{ct}.insulation.nc')
ins_count = ins_count.sel({'bin': (ins_count['bin_chrom']!='chrX')})
ins_count['ratio'] = (ins_count.sel({'type':'inter'})['__xarray_dataarray_variable__'] / ins_count.sel({'type':'intra'}))['__xarray_dataarray_variable__']
ins_count

<xarray.Dataset>
Dimensions:                        (bin: 98520, cell: 3, type: 2)
Coordinates:
    bin_chrom                      (bin) object 'chr1' 'chr1' ... 'chr19'
  * bin                            (bin) int32 0 1 2 3 ... 98517 98518 98519
    bin_start                      (bin) int32 ...
    bin_end                        (bin) int32 ...
  * cell                           (cell) object 'Astro-NT_NN.18mo' ... 'Astr...
  * type                           (type) object 'inter' 'intra'
Data variables:
    __xarray_dataarray_variable__  (cell, bin, type) float64 ...
    ratio                          (cell, bin) float64 1.0 0.0 0.0 ... 0.0 0.0

In [12]:
ins = ins_count['ratio'].to_pandas().loc[leg]
ins.shape

(3, 98520)

In [13]:
binall = ins_count[['bin_chrom', 'bin_start', 'bin_end']].to_pandas()
binall.columns = binall.columns.str.split('_').str[1]
binall.index = binall['chrom'] + '_' + (binall['start'] // res).astype(str)

In [14]:
chi2sc, fdr_sc = diff_bound(bound_count_ct.values, cell_count_ct.values)
ave = np.mean(chi2sc[chi2sc>0])
stdev = np.std(chi2sc[chi2sc>0])
binall['chi2filter'] = (((chi2sc - ave) / stdev)>norm.isf(0.025))

In [15]:
binall['ins_lm'] = 0
for xx in leg:
    sel = []
    for c in chrom_sizes.index:
        idx = np.where(binall['chrom']==c)[0]
        if len(idx)>0:
            data = -ins.loc[xx, idx]
            peaks, _ = find_peaks(data, distance=5)
            sel.append(idx.min() + peaks)
    sel = np.concatenate(sel)
    binall.loc[binall.index[sel], 'ins_lm'] = 1

binall['probdiff'] = (bound_prob_ct.max(axis=1) - bound_prob_ct.min(axis=1)).values
binall['chi2_sc'] = chi2sc.copy()
binall['insfc'] = (ins.max(axis=0)+0.01 / ins.min(axis=0)+0.01).values

In [16]:
sel = []
thres = np.min(chi2sc[fdr_sc<1e-3])
for c in chrom_sizes.index:
    idx = np.where(binall['chrom']==c)[0]
    if len(idx)>0:
        data = chi2sc[idx]
        peaks, _ = find_peaks(data, height=thres, distance=5)
        sel.append(idx.min() + peaks)
        
sel = np.concatenate(sel)

binall['diff_sc'] = 0
binall.loc[binall.index[sel], 'diff_sc'] = 1
binall.loc[:, binall.dtypes=='category'] = binall.loc[:, binall.dtypes=='category'].astype(str)
binall.to_hdf(f'{ct}_bin_stats.hdf', key='data')

In [17]:
binall

,chrom,start,end,chi2filter,ins_lm,probdiff,chi2_sc,insfc,diff_sc
chr1_0,chr1,0,25000,False,0,0.000000,0.000000,1.020000,0
chr1_1,chr1,25000,50000,False,0,0.000000,0.000000,inf,0
chr1_2,chr1,50000,75000,False,0,0.000000,0.000000,inf,0
chr1_3,chr1,75000,100000,False,0,0.000000,0.000000,inf,0
chr1_4,chr1,100000,125000,False,0,0.000000,0.000000,inf,0
...,...,...,...,...,...,...,...,...,...
chr19_2453,chr19,61325000,61350000,False,0,0.003236,1.002162,0.449829,0
chr19_2454,chr19,61350000,61375000,False,0,0.000000,0.000000,1.318361,0
chr19_2455,chr19,61375000,61400000,False,0,0.000000,0.000000,inf,0
chr19_2456,chr19,61400000,61425000,False,0,0.000000,0.000000,inf,0
